# Generador de personas aleatorias

In [1]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import unicodedata
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder

In [136]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
ciudades = pd.read_csv('./catalogos/ciudades.csv', encoding = 'UTF-16')

def quitar_acentos(s):
	return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def generate_random_data(lat, lon, num_rows = 5):
	data = []
	for _ in range(num_rows):
		hex1 = '%012x' % rm.randrange(16 ** 12)
		flt = float(rm.randint(0, 100))
		dec_lat = rm.random() / 100
		dec_lon = rm.random() / 100
		data.append((hex1.lower(), flt, lon + dec_lon, lat + dec_lat))
	return data

def save_random_data(lat, lon, num_rows = 5, file_name = './conjunto_datos/random_lat_lon.prueba.csv'):
	with open(file_name, 'w') as output:
		for _ in range(num_rows):
			hex1 = '%012x' % rm.randrange(16 ** 12)
			flt = float(rm.randint(0, 100))
			dec_lat = rm.random() / 100
			dec_lon = rm.random() / 100
			output.write('%s,%.1f,%.6f,%.6f\n' % (hex1.lower(), flt, lon + dec_lon, lat + dec_lat))

In [131]:
fake = Faker(['es_MX'])
personas = []
for _ in range(5):
	sexo = rm.choice(['H', 'M'])
	# generar tipo de sangre y asignar probabilidad a cada valor para que salga
	tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
	tipo_sangre = tipo_sangre[0]

	fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
	# calcular edad en base a la fecha de nacimiento
	dia_actual = dt.date.today()
	edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

	# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
	# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
	# mucha gente deja de crecer a los 20
	margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

	# dependiendo del sexo se generan nombres femeninos o masculinos
	# y estaturas y pesos dentro de su margen correspondiente
	if sexo == 'H':
		titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
		nombre = rm.choice(nombres['h'])
		altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
		peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
	else:
		titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
		nombre = rm.choice(nombres['m'])
		altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
		peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

	apellido_paterno = fake.last_name()
	apellido_materno = fake.last_name()

	# Generar un nombre de usuario para un email
	switcher = {
		1: fake.user_name(),
		# aperez
		2: f'{nombre[0]}{apellido_paterno}',
		# alonso_perez
		3: f'{nombre}_{apellido_paterno}',
		# aps70
		4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
		# aps1970
		5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
		# alonsoperezsoltero
		6: f'{nombre}{apellido_paterno}{apellido_materno}'
	}
	rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
	usuario = switcher.get(rand_int, 'usuario')
	# quitar espacios o reemplazarlos por un _
	usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
	usuario = quitar_acentos(usuario)

	subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
	correo = usuario + f'@example{subdominio}'

	id_estado = rm.choice(ciudades['id_estado'].unique())
	ct_ciudad = ciudades[ciudades['id_estado'] == id_estado].sample()

	ciudad = ct_ciudad['ciudad'].values[0]
	estado = ct_ciudad['estado'].values[0]
	tel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))

	direccion = fake.street_address()

	direcc_calle = fake.street_name()
	direcc_numero = fake.building_number()
	direcc_cp = fake.postcode()

	profesion = profesiones.sample().values[0][0] if edad >= 18 else None

	# si el catalogo ya tiene coordenadas puestas, tomar esos valores
	if pd.notna(ct_ciudad['latitud'].values[0]) and pd.notna(ct_ciudad['longitud'].values[0]) and pd.notna(ct_ciudad['zona_horaria'].values[0]):
		lat = ct_ciudad['latitud'].values[0]
		lon = ct_ciudad['longitud'].values[0]
		timezone = ct_ciudad['zona_horaria'].values[0]
	else:
		try:
			# obtener latitud y longitud segun la ciudad
			geolocator = Nominatim(user_agent = 'geoapiExercises')
			location = geolocator.geocode(f'{ciudad}, {estado}, Mexico', language = 'es-MX')
			if location:
				lat = location.latitude
				lon = location.longitude
				# obtener zona horaria segun latitud y longitud
				obj = TimezoneFinder()
				timezone = obj.timezone_at(lat = lat, lng = lon)

				# for _ in range(5):
				# 	dec_lat = rm.random() / 100
				# 	dec_lon = rm.random() / 100
				# 	dirr = geolocator.reverse((lat + dec_lat, lon + dec_lon), language = 'es', addressdetails = True)
				# 	print(dirr)

				# guardar coordenadas en el catalogo
				# para que a la otra no haya que usar geocode
				ct_ciudad_index_row = ct_ciudad.index.values[0]
				ciudades.loc[ct_ciudad_index_row, 'latitud'] = lat
				ciudades.loc[ct_ciudad_index_row, 'longitud'] = lon
				ciudades.loc[ct_ciudad_index_row, 'zona_horaria'] = timezone
				ciudades.to_csv('./catalogos/ciudades.prueba.csv', index = False)
			else:
				raise BaseException(f'(idx {_}) No se encontro latitud ni longitud de {ciudad}, {estado}')
		except BaseException as be:
			print('\033[32mException:\033[0m')
			print(be)
			lat = lon = np.NaN
			timezone = None

	personas.append({
		'titulo': titulo,
		'nombre': nombre,
		'apellido_paterno': apellido_paterno,
		'apellido_materno': apellido_materno,
		'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
		'sexo': sexo,
		'tipo_sangre': tipo_sangre,
		'altura': altura,
		'peso': peso,
		'correo': correo,
		'tel': tel,
		# 'calle': '',
		# 'numero': '',
		# 'cp': '',
		'ciudad': ciudad,
		'estado': estado,
		'lat': lat,
		'lon': lon,
		'zona_horaria': timezone,
		'profesion': profesion,
		'color_fav': fake.safe_color_name(),
	})
pd.DataFrame(personas)

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,tipo_sangre,altura,peso,correo,tel,ciudad,estado,lat,lon,zona_horaria,profesion,color_fav
0,None,Homero,Esparza,Orozco,1926-08-04,H,A+,182.73,94.17,heo26@example.io,(746)4215975,Metlaltoyuca,Puebla,20.735556,-97.851944,America/Mexico_City,Oficial de conservación de naturaleza,lime
1,Dra.,Rocío,Luevano,Meraz,1929-02-28,M,O-,170.34,47.24,rocio_luevano@example.com,(443)7237516,Morelia,Michoacan,19.702712,-101.192382,America/Mexico_City,Librero,purple
2,Dra.,María Teresa,Vargas,Villalpando,1998-04-13,M,O+,164.64,64.78,mvargas@example.com,(314)2768842,Camotlán de Miraflores,Colima,19.220190,-104.234996,America/Mexico_City,Horticultor,purple
3,Sra.,Judith,Palomino,Alva,1942-12-28,M,A+,173.04,69.43,jpa42@example.io,(323)9098180,Santiago Ixcuintla,Nayarit,21.959108,-105.551989,America/Mazatlan,Tesorero corporativo,gray
4,Mtro.,Esteban,Negrete,Solorzano,1992-10-07,H,A+,179.86,82.88,enegrete@example.net,(712)2477163,Jiquipilco,Estado de Mexico,19.591632,-99.631174,America/Mexico_City,Científico biomédico,white


In [117]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query

personas = pd.read_csv(url)
personas.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
62,male,Mr,Nicklas,Johansen,1217,Frederiksborgvej,Nørrebro,Syddanmark,Denmark,27372,33.3072,157.7187,+4:30,Kabul,nicklas.johansen@example.com,1965-09-14T23:16:38.308Z,57,77401924,91068794
60,female,Miss,Embla,Skjæret,1537,Eikeveien,Levanger,Buskerud,Norway,4642,22.6395,116.0633,-1:00,"Azores, Cape Verde Islands",embla.skjaeret@example.com,1973-10-08T09:58:07.762Z,49,68550908,45269636
25,male,Mr,Eli,Singh,1882,Gladstone Road,Greymouth,Waikato,New Zealand,79550,77.6386,151.6528,-5:00,"Eastern Time (US & Canada), Bogota, Lima",eli.singh@example.com,1971-04-15T11:37:58.102Z,51,(547)-178-5107,(026)-115-1175
85,female,Mrs,Freja,Madsen,2714,Strandvej,Randers Nv,Syddanmark,Denmark,26549,-20.2449,-171.2555,+9:00,"Tokyo, Seoul, Osaka, Sapporo, Yakutsk",freja.madsen@example.com,1966-09-29T02:28:48.384Z,56,27780652,28807534
80,male,Mr,Jar,Kelley,7197,Brown Terrace,Killeen,Alaska,United States,41584,-0.2664,-67.0959,0:00,"Western Europe Time, London, Lisbon, Casablanca",jar.kelley@example.com,1968-03-22T05:11:36.035Z,54,(775)-963-0793,(294)-398-8814
48,male,Mr,Xavier,Jones,7120,Pine Rd,Georgetown,Saskatchewan,Canada,Q1O 7V7,66.1318,40.9410,+6:00,"Almaty, Dhaka, Colombo",xavier.jones@example.com,1996-02-11T08:13:20.101Z,26,664-304-9491,520-836-3330
26,female,Mademoiselle,Mia,Leroy,2148,Rue Baraban,Oulens-Sous-Echallens,Thurgau,Switzerland,3884,42.3597,177.8986,+6:00,"Almaty, Dhaka, Colombo",mia.leroy@example.com,1996-08-03T22:44:45.110Z,26,076 331 29 17,078 480 88 17
36,male,Mr,Mehmet,Ayaydın,8779,Atatürk Sk,Kastamonu,Muğla,Turkey,62397,10.4526,-56.7678,-7:00,Mountain Time (US & Canada),mehmet.ayaydin@example.com,1945-07-30T13:17:07.251Z,77,(653)-673-2668,(491)-149-0570
